In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #Data Visualization 
import seaborn as sns  #Python library for Vidualization
from sklearn.cluster import KMeans

max = 9223372036854775807

In [2]:
# defining calculation functions

def calcBA(pruns, pinnings, pnotout):
    ba = pruns / (pinnings - pnotout)
    return ba


def calcBS(pruns, pbfaced):
    bs = pruns / pbfaced
    return bs


def calcMRA(pinnings, pnotout, pbfaced):
    mra = (pinnings - pnotout) / pbfaced
    return mra


def calcBRPI(pfours, psixes, pinnings):
    brpi = (4*pfours + 6*psixes) / pinnings
    return brpi


def calcBatOutRate(pinnings, pnotout, pbfaced):
    outrate = pd.Series(np.array([max])) if float(pbfaced) == 0 else (pinnings - pnotout) / pbfaced
    return outrate

#         BA = calcBA(p['runs'], p['innings'], p['not_out'])
#         BS = calcBS(p['runs'], p['ball_faced'])
#         MRA = calcMRA(p['innings'], p['not_out'], p['ball_faced'])
#         BRPI = calcBRPI(p['fours'], p['sixes'], p['innings'])


def calcGenBatAvg(truns, twickets):
    return truns / twickets


def calcGenBatOutRate(twickets, tbfaced):
    return twickets / tbfaced


def calcGenStrikeRate(truns, tbfaced):
    return truns / tbfaced


def calcBatAGR(tpRuns, tpBallfaced, tpOutRate, gBatAvg, gStrikeRate, gOutRate):
    agr = ((tpRuns - gStrikeRate * tpBallfaced) + gBatAvg * tpBallfaced * (gOutRate - tpOutRate))
#     d = np.array([132])
#     tm = pd.Series(np.array([0]))
    return agr


def calcBatRankingIndex(agr, gBatAvg):
    return agr / (10 * gBatAvg)


In [3]:
# defining bpwlling related calculation functions

# for each player

def calcBowlAvg(runsGiven, wicketsTaken):
    return runsGiven / wicketsTaken


def calcBowlSR(ballsBowled, wicketsTaken):
    return ballsBowled / wicketsTaken


def calcBowlER(runsGiven, oversBowled):
    return runsGiven / oversBowled


def calcBowlOutRate(wicketsTaken, ballsBowled):
    outr = pd.Series(np.array([0])) if float(ballsBowled) == 0 else wicketsTaken / ballsBowled
    return outr

# generic tournament parameters

def calcGenBowlAvg(totalRunsGiven, totalWicketsTaken):
    return totalRunsGiven / totalWicketsTaken


def calcGenBowlOutRate(totalWicketsTaken, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
    return totalWicketsTaken / balls


def calcGenBowlSR(totalRunsGiven, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
    return  totalRunsGiven / balls


def calcBowlAGR(tpRunsGiven, tpOvers, tpOutrate, gBowlAvg, gBowlSR, gBowlOutrate ):
    balls = int(tpOvers) * 6 + 10 * (tpOvers - int(tpOvers))
    bowlagr = (gBowlSR * balls - tpRunsGiven) + gBowlAvg * balls * (tpOutrate - gBowlOutrate)
    
#     print(bowlagr[0])
    if tpOvers[0] == 0.1:
        print('-------------------------------')
        print('tpRunsGiven  :',tpRunsGiven[0]) 
        print('tpballs      :', balls[0])
        print('tpOutrate    :', tpOutrate[0])
        print('gBowlAvg     :', gBowlAvg[0]) 
        print('gBowlSR      :', gBowlSR[0])
        print('gBowlOutrate :',  gBowlOutrate[0])
        print('-------------------------------')
        print('bowlAGR      : ', bowlagr)
        
    return bowlagr


def calcBowlRI(bowlAGR, gBowlAvg):
    bri = bowlAGR / (10 * gBowlAvg)
    
    if bri[0] == 0:
        print('-------------------------------')
        print('bowlagr  : ', bowlAGR[0] )
        print('gbowlAvg : ', gBowlAvg[0])
        print('-------------------------------')
    
    return bowlAGR / (10 * gBowlAvg)

In [7]:
df = pd.read_csv("newPlayers.csv")
print(df.columns)

Index(['series_code', 'match_code', 'team', 'pid', 'name', 'notout', 'runs',
       'ballfaced', 'fours', 'sixes', 'overs', 'maiden_overs', 'runs_given',
       'wickets_taken'],
      dtype='object')


In [8]:
df['ballfaced'].replace({0 : 1}, inplace=True)
df['overs'].replace({0.0 : 0.1}, inplace=True)
# df[df['ballfaced'] == 1]

In [11]:
series = list(dict.fromkeys(df["series_code"]).keys())
# series.sort(reverse=True)

# trecord : dataframe for tournament overall performance
# precord : dataframe for player specific performance in tournament
# tplayer: dataframe for recording total runs, ballfaced etc for player

# seriespdata = pd.DataFrame()
trecord = pd.DataFrame()
precord = pd.DataFrame()
tplayer = pd.DataFrame()
pindexes = pd.DataFrame()

tournamentPerformance = dict()

for sr in series[:]:
    tdf = df[df["series_code"] == sr]
 
    ttotal = tdf.drop(['series_code', 'match_code', 'pid', 'name'], axis = 1)
    
    ttotal = pd.DataFrame(ttotal.sum(axis = 0))
    ttotal = ttotal.swapaxes('index', 'columns')
    ttotal.insert(0, 'series_code', value = sr)
    
    gBatAvg = calcGenBatAvg(ttotal['runs'], ttotal['wickets_taken'])
    gBatOutRate = calcGenBatOutRate(ttotal['wickets_taken'], ttotal['ballfaced'])
    gStrikeRate = calcGenStrikeRate(ttotal['runs'], ttotal['ballfaced'])
    
    gBowlAvg = calcBowlAvg(ttotal['runs_given'], ttotal['wickets_taken'])
    gBowlOutrate = calcGenBowlOutRate(ttotal['wickets_taken'], ttotal['overs'])
    gBowlSR = calcGenBowlSR(ttotal['runs_given'], ttotal['overs'])
    
    tp = {
        'series_code' : sr, 
        'gBatAvg'     : gBatAvg[0], 
        'gBatOutRate' : gBatOutRate[0], 
        'gStrikeRate' : gStrikeRate[0],
        'gBowlAvg'    : gBowlAvg[0], 
        'gBowlOutrate' : gBowlOutrate[0], 
        'gBowlSR'     : gBowlSR[0]
    }
    
    trecord = trecord.append(tp, ignore_index=True)
    
    
#     print((tp))
# print(len(series))

    players = list()
    for pl in set(tdf["pid"]):
        if pl > 0:
            players.append(pl)
        
#     print('current ',players)
    
    for player in players[:]:
        
        
        ptdf = tdf[tdf['pid'] == player].drop(['series_code', 'match_code', 'pid', 'name'], axis = 1)
        
        pinnings = len(ptdf)
        p = pd.DataFrame(ptdf.sum(axis = 0))
        p = p.swapaxes('index', 'columns')
        
#         print(p)
        tplayer = tplayer.append(p)
    
        batOutRate = calcBatOutRate(pinnings, p['notout'], p['ballfaced'])
        bowlOutRate = calcBowlOutRate(p['wickets_taken'], p['overs'])
        
#         p.insert(0, 'innings' ,value = pinnings)
#         p.insert(0, 'pid' ,value = player)
#         p.insert(0, 'series_code' ,value = sr)
#         p.insert(4, 'outrate' ,value = outRate)

        batAGR = calcBatAGR(p['runs'], p['ballfaced'], batOutRate, gBatAvg, gStrikeRate, gBatOutRate)
        batRI = calcBatRankingIndex(batAGR, gBatAvg)
        
        bowlAGR = calcBowlAGR(p['runs_given'], p['overs'], bowlOutRate, gBowlAvg, gBowlSR, gBowlOutrate)
        bowlRI = calcBowlRI(bowlAGR, gBowlAvg)
        
        pRI = {
            'series_code' : sr,
            'pid'    : player,
            'batAGR' : batAGR[0],
            'batRI'  : batRI[0],
            'bowlAGR': bowlAGR[0],
            'bowlRI' : bowlRI[0]
        }
        
        pindexes = pindexes.append(pRI, ignore_index = True)
#         print(type(batAGR))
#         print(batOutRate[0], bowlOutRate[0])
#         print('batAGR ', batAGR[0], '                        batRI ', batRI[0])
#         print('bowlAGR ', bowlAGR[0], '                       bowlRI ', bowlRI[0])
#         print()
        

-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.72173058013766
gBowlSR      : 0.7612718269481247
gBowlOutrate : 0.026505082095386608
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
---

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.72173058013766
gBowlSR      : 0.7612718269481247
gBowlOutrate : 0.026505082095386608
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.72173058013766
gBowlSR      : 0.7612718269481247
gBowlOutrate : 0.026505082095386608
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
tpRunsGive

-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.72173058013766
-------------------------------
----------

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.323223570190642
gBowlSR      : 0.8018891069675987
gBowlOutrate : 0.028312070657506773
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.323223570190642
gBowlSR      : 0.8018891069675987
gBowlOutrate : 0.028312070657506773
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowl

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.323223570190642
gBowlSR      : 0.8018891069675987
gBowlOutrate : 0.028312070657506773
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.323223570190642
gBowlSR      : 0.8018891069675987
gBowlOutrate : 0.028312070657506773
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.323223570190642
-------------------------------
-------------------------------
bowl

...

-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.326674500587544
gBowlSR      : 0.780192871610356
gBowlOutrate : 0.025726291604944707
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587

-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.326674500587544
gBowlSR      : 0.780192871610356
gBowlOutrate : 0.025726291604944707
-------------------------------
b

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.326674500587544
gBowlSR      : 0.780192871610356
gBowlOutrate : 0.025726291604944707
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.326674500587544
gBowlSR      : 0.780192871610356
gBowlOutrate : 0.025726291604944707
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.326674500587544
-------------------------------
-------------------------------
tpRuns

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.551761517615176
gBowlSR      : 0.789242639898087
gBowlOutrate : 0.027642520038953084
-------------------------------
bowlAGR      :  0   -1.11022e-16
dtype: object
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.551761517615176
gBowlSR      : 0.789242639898087
gBowlOutrate : 0.027642520038953084
-------------------------------
bowlAGR      :  0   -1.11022e-16
dtype: object
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.551761517615176
gBowlSR      : 0.789242639898087
gBowlOutrate : 0.027642520038953084
-------------------------------
bowlAGR      :  0   -1.11022e-16
dtype: object
-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 28.551761517615176
gBowlSR      : 0.789242639898087
gBowlOutrate : 0.027642520038953084
---

-------------------------------
tpRunsGiven  : 0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 31.934782608695652
gBowlSR      : 0.846685878962536
gBowlOutrate : 0.026512968299711816
-------------------------------
bowlAGR      :  0    0
dtype: object
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.934782608695652
-------------------------------


In [12]:
pindexes.head()

,batAGR,batRI,bowlAGR,bowlRI,pid,series_code
0,195.837758,0.706041,241.427551,0.840575,8198.0,2364.0
1,-79.212389,-0.285579,341.991150,1.190705,10249.0,2364.0
2,63.787611,0.229969,0.000000,0.000000,10250.0,2364.0
3,-104.687316,-0.377422,476.141436,1.657774,10251.0,2364.0
4,-60.687316,-0.218792,754.035464,2.625313,8204.0,2364.0


In [13]:
pindexes.to_csv('newPlayerIR01.csv', index = False)
